# ---------------------------------------------
# 🏥 Synthetic Hospital Performance Dataset Generator
# ---------------------------------------------
# This script generates 5 synthetic datasets for a hospital performance dashboard.
# The datasets include information about patients, doctors, departments, and their performance metrics.
# The generated datasets are saved in CSV format for loading into Power BI.
# ---------------------------------------------

In [2]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import os

In [3]:
# Initialize Faker
fake = Faker()

# Set a random seed for reproducibility
np.random.seed(42)
random.seed(42)

# 📁 Create output folder
output_folder = "Healthcare_Datasets"
os.makedirs(output_folder, exist_ok=True)

In [35]:
# -----------------------------------------
# 1. Generate Department Table
# -----------------------------------------

departments = pd.DataFrame({
    "DepartmentID": range(1, 6),
    "DepartmentName": ["Cardiology", "Neurology", "Orthopedics", "Pediatrics", "Oncology"],
    "BedCapacity" : np.random.randint(50, 120, 5)
})
departments.to_csv(f"{output_folder}/departments.csv", index=False)
print("Department Table Created")
print(departments.head())

Department Table Created
   DepartmentID DepartmentName  BedCapacity
0             1     Cardiology           55
1             2      Neurology           54
2             3    Orthopedics           94
3             4     Pediatrics           65
4             5       Oncology           66


In [36]:
# -----------------------------------------
# 2. Generate Staff Profile Table
# -----------------------------------------

staff_profiles = pd.DataFrame({
    "StaffID": range(1, 11),
    "Name" : [fake.name() for _ in range(10)],
    "DepartmentID" : np.random.choice(departments["DepartmentID"], 10),
    "Role" : np.random.choice(["Doctor", "Nurse", "Technician"], 10),
    "ExperienceYears" : np.random.randint(1, 20, 10),
    "Photo" : [fake.image_url() for _ in range(10)]
})
staff_profiles.to_csv(f"{output_folder}/staff_profiles.csv", index=False)
print("Staff Profile Table Created")
print(staff_profiles.head())

Staff Profile Table Created
   StaffID             Name  DepartmentID        Role  ExperienceYears  \
0        1    Joyce Robbins             3       Nurse                4   
1        2   Willie Douglas             5  Technician               17   
2        3  Jenna Valentine             3       Nurse               10   
3        4      Samuel Lowe             3  Technician                1   
4        5    Chelsea Casey             3      Doctor               11   

                             Photo  
0    https://picsum.photos/152/614  
1    https://dummyimage.com/160x31  
2  https://placekitten.com/731/790  
3    https://picsum.photos/856/435  
4    https://picsum.photos/258/161  


In [37]:
# -----------------------------------------
# 3. Generate Patient Table
# -----------------------------------------

patients = pd.DataFrame({
    "PatientID": range(1, 1001),
    "Name": [fake.name() for _ in range(1000)],
    "Gender" : np.random.choice(["Male", "Female"], 1000),
    "Age" : np.random.randint(1, 90, 1000)
})
patients.to_csv(f"{output_folder}/patients.csv", index=False)
print("Patients Table Created")
print(patients.head())

Patients Table Created
   PatientID                 Name  Gender  Age
0          1     Margaret Bradley    Male    1
1          2      Craig Fernandez    Male   34
2          3     Melanie Matthews    Male   85
3          4  Danielle Mccullough  Female   10
4          5  Stephanie Henderson    Male    5


In [38]:
# -----------------------------------------
# 4. Hospital Stay Table
# -----------------------------------------

hospital_stay = pd.DataFrame({
    "StayID" : range(1, 1501),
    "PatientID" : np.random.choice(patients["PatientID"], 1500),
    "DepartementID" : np.random.choice(departments["DepartmentID"], 1500),
    "DoctorID" : np.random.choice(staff_profiles["StaffID"], 1500),
    "AdmissionDate" : pd.to_datetime(np.random.choice(pd.date_range("2023-01-01", "2023-12-31"), 1500)),
})

# ✅ Add Discharge, Satisfaction, Readmission, and Severity Level
hospital_stay["DischargeDate"] = hospital_stay["AdmissionDate"] + pd.to_timedelta(np.random.randint(1, 10, 1500), unit="D")
hospital_stay["SatisfactionScore"] = np.round(np.random.uniform(3.5, 5, 1500), 1)
hospital_stay["Readmission"] = np.random.choice(["Yes", "No"], 1500, p=[0.15, 0.85])
hospital_stay["SeverityLevel"] = np.random.choice(["Low", "Medium", "High", "Critical"], 1500, p=[0.4, 0.3, 0.2, 0.1])

# ✅ Add Cost Column based on Severity Level
severitylevel_cost_mapping = {
    "Low" : lambda: np.round(np.random.normal(1500, 300), 2),
    "Medium" : lambda: np.round(np.random.normal(3500, 700), 2),
    "High" : lambda: np.round(np.random.normal(6000, 1200), 2),
    "Critical" : lambda: np.round(np.random.normal(10000, 2000), 2)
}
hospital_stay["Cost"] = hospital_stay["SeverityLevel"].apply(lambda s: severitylevel_cost_mapping[s]())

diagnosis_list = [
    "Hypertension", "Diabetes", "Pneumonia", "Asthma", "Fracture",
    "Migraine", "Cancer", "Infection", "Appendicitis", "Heart Disease"
]
hospital_stay["Diagnosis"] = np.random.choice(diagnosis_list, size=len(hospital_stay))

hospital_stay.to_csv(f"{output_folder}/hospital_stay.csv", index=False)
print("Hospital Stay Table Created")
print(hospital_stay.head())

Hospital Stay Table Created
   StayID  PatientID  DepartementID  DoctorID AdmissionDate DischargeDate  \
0       1        515              5         4    2023-07-03    2023-07-09   
1       2        471              4         6    2023-04-25    2023-04-26   
2       3        239              3         6    2023-05-04    2023-05-11   
3       4        289              5         8    2023-08-26    2023-08-29   
4       5        368              4         8    2023-04-18    2023-04-26   

   SatisfactionScore Readmission SeverityLevel     Cost      Diagnosis  
0                4.2          No          High  6333.13       Migraine  
1                4.7          No           Low  1683.77       Migraine  
2                4.9          No        Medium  3526.87  Heart Disease  
3                5.0          No           Low  1927.62         Asthma  
4                4.9          No           Low  1717.63   Appendicitis  


In [53]:
# -----------------------------------------
# 5. Generate Department Daily Table
# -----------------------------------------

flu_sensitive_departments = ["Pediatrics", "Oncology"]

department_daily = []
for _, row in departments.iterrows():
    previous_day_occupancy = 10  # Start with a base occupancy

    for date in pd.date_range("2023-01-01", "2023-12-31"):
        flu_season = date.month in [1, 2, 12]
        is_flu_sensitive = row.DepartmentName in flu_sensitive_departments

        # Base admissions
        admissions = random.randint(0, int(row.BedCapacity * 0.2))

        # Increase admissions if flu season + sensitive department
        if flu_season and is_flu_sensitive:
            admissions += random.randint(0, int(row.BedCapacity * 0.2))

        discharges = random.randint(0, admissions)

        # Adjust bed occupancy based on season and department
        if flu_season and is_flu_sensitive:
            bed_occupancy = random.randint(int(row.BedCapacity * 0.7), row.BedCapacity)
        else:
            bed_occupancy = random.randint(int(row.BedCapacity * 0.4), int(row.BedCapacity * 0.85))

        department_daily.append({
            "DepartmentID": row.DepartmentID,
            "Date": date,
            "Admissions": admissions,
            "Discharges": discharges,
            "BedOccupancy": bed_occupancy,
        })

        previous_day_occupancy = bed_occupancy


department_daily_df = pd.DataFrame(department_daily)
department_daily_df.to_csv(f"{output_folder}/department_daily.csv", index=False)
print("Department Daily Table Created") 
print(department_daily_df.head())

Department Daily Table Created
   DepartmentID       Date  Admissions  Discharges  BedOccupancy
0             1 2023-01-01           8           0            45
1             1 2023-01-02           5           1            35
2             1 2023-01-03           3           2            36
3             1 2023-01-04          10           1            44
4             1 2023-01-05           6           3            38


In [29]:
# ✅ Print confirmation message
print("✅ All datasets have been successfully generated and saved in:", output_folder)

✅ All datasets have been successfully generated and saved in: Healthcare_Datasets
